# Making a Flower Classifier with VGG16

### Loading the VGG16 Model

In [1]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

#Loads the VGG16 model 
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

Using TensorFlow backend.


58892288/58889256 [==============================] - 37s 1us/step


### Inpsecting each layer

In [2]:
# Let's print our layers 
for (i,layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


### Let's freeze all layers except the top 4 

In [1]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

# Re-loads the VGG16 model without the top or FC layers
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in vgg16.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

Using TensorFlow backend.
W0822 17:24:42.452999 10336 deprecation_wrapper.py:119] From D:\development\anaconda3\envs\dlcv\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0822 17:24:42.721999 10336 deprecation_wrapper.py:119] From D:\development\anaconda3\envs\dlcv\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0822 17:24:42.825999 10336 deprecation_wrapper.py:119] From D:\development\anaconda3\envs\dlcv\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0822 17:24:43.049000 10336 deprecation_wrapper.py:119] From D:\development\anaconda3\envs\dlcv\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0822 17:24:4

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


### Let's make a function that returns our FC Head

In [2]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

### Let's add our FC Head back onto VGG

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 17

FC_Head = addTopModel(vgg16, num_classes)

model = Model(inputs=vgg16.input, outputs=FC_Head)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### Loading our Flowers Dataset

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = './17_flowers/train'
validation_data_dir = './17_flowers/validation'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


### Training our top layers

In [6]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("D:/development/DeepLearningCV/Trained Models/flowers_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170
epochs = 3
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)



W0822 17:27:40.999000 10336 deprecation_wrapper.py:119] From D:\development\anaconda3\envs\dlcv\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0822 17:27:41.263000 10336 deprecation.py:323] From D:\development\anaconda3\envs\dlcv\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/3
74/74 [==============================] - 454s 6s/step - loss: 11.8475 - acc: 0.1166 - val_loss: 2.4457 - val_acc: 0.4200

Epoch 00001: val_loss improved from inf to 2.44567, saving model to D:\development\DeepLearningCV/Trained Models/flowers_vgg.h5
Epoch 2/3
74/74 [==============================] - 472s 6s/step - loss: 2.3029 - acc: 0.3029 - val_loss: 0.9795 - val_acc: 0.7200

Epoch 00002: val_loss improved from 2.44567 to 0.97946, saving model to D:\development\DeepLearningCV/Trained Models/flowers_vgg.h5
Epoch 3/3
74/74 [==============================] - 356s 5s/step - loss: 1.7705 - acc: 0.4420 - val_loss: 1.3048 - val_acc: 0.6600

Epoch 00003: val_loss did not improve from 0.97946


OSError: Unable to create file (unable to open file: name = '/home/deeplearningcv/DeepLearningCV/Trained Models/flowers_vgg.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [7]:
model.save("D:/development/DeepLearningCV/Trained Models/flowers_vgg.h5")

In [9]:
class_labels=validation_generator.class_indices
target_names = list(class_labels.keys())
target_labels = list(class_labels.values())
print(target_names)
print(target_labels)

['bluebell', 'buttercup', 'colts_foot', 'cowslip', 'crocus', 'daffodil', 'daisy', 'dandelion', 'fritillary', 'iris', 'lily_valley', 'pansy', 'snowdrop', 'sunflower', 'tigerlily', 'tulip', 'windflower']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


## Can we speed this up?
#### Let's try re-sizing the image to 64 x 64

In [13]:
from keras.applications import VGG16

# Setting the input size now to 64 x 64 pixel 
img_rows = 64
img_cols = 64 

# Re-loads the VGG16 model without the top or FC layers
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in vgg16.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


### Let's create our new model using an image size of 64 x 64

In [14]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = './17_flowers/train'
validation_data_dir = './17_flowers/validation'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

# Re-loads the VGG16 model without the top or FC layers
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Freeze layers
for layer in vgg16.layers:
    layer.trainable = False
    
# Number of classes in the Flowers-17 dataset
num_classes = 17

FC_Head = addTopModel(vgg16, num_classes)

model = Model(inputs=vgg16.input, outputs=FC_Head)

print(model.summary())

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________

### Training using 64 x 64 image size is MUCH faster!

In [7]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
                   
checkpoint = ModelCheckpoint("D:/development/DeepLearningCV/Trained Models/flowers_vgg_64.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.00001)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]

# Note we use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.0001),
              metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170
epochs = 25
batch_size = 32

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

model.save("/home/deeplearningcv/DeepLearningCV/Trained Models/flowers_vgg_64.h5")

Epoch 1/25
37/37 [==============================] - 19s 510ms/step - loss: 0.5119 - acc: 0.8328 - val_loss: 0.7950 - val_acc: 0.7800

Epoch 00001: val_loss improved from inf to 0.79499, saving model to /home/deeplearningcv/DeepLearningCV/Trained Models/flowers_vgg_64.h5
Epoch 2/25
37/37 [==============================] - 18s 482ms/step - loss: 0.4786 - acc: 0.8306 - val_loss: 1.0675 - val_acc: 0.6600

Epoch 00002: val_loss did not improve from 0.79499
Epoch 3/25
37/37 [==============================] - 18s 496ms/step - loss: 0.4051 - acc: 0.8615 - val_loss: 0.9847 - val_acc: 0.7800

Epoch 00003: val_loss did not improve from 0.79499
Epoch 4/25
37/37 [==============================] - 18s 497ms/step - loss: 0.4178 - acc: 0.8733 - val_loss: 0.7770 - val_acc: 0.8000

Epoch 00004: val_loss improved from 0.79499 to 0.77698, saving model to /home/deeplearningcv/DeepLearningCV/Trained Models/flowers_vgg_64.h5
Epoch 5/25
37/37 [==============================] - 19s 505ms/step - loss: 0.5334 - 

In [ ]:
###Load classifer

In [8]:
from keras.models import load_model
classifier = load_model('D:/development/DeepLearningCV/Trained Models/flowers_vgg.h5')

In [ ]:
### Testing our classifer on some test images

In [23]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

flower_dict = {
    "[0]": "bluebell ", 
    "[1]": "buttercup",
     "[2]": "colts_foot",
     "[3]": "cowslip",
     "[4]": "crocus ",
     "[5]": "daffodil",
     "[6]": "daisy",
     "[7]": "dandelion",
     "[8]": "fritillary",
     "[9]": "iris",
     "[10]": "lily_valley ",
     "[11]": "pansy",
     "[12]": "snowdrop",
     "[13]": "sunflower",
     "[14]": "tigerlily",
     "[15]": "tulip",
     "[16]": "windflower"
}
flower_dict_n = {
    "n0": "bluebell ", 
    "n1": "buttercup",
     "n2": "colts_foot",
     "n3": "cowslip",
     "n4": "crocus ",
     "n5": "daffodil",
     "n6": "daisy",
     "n7": "dandelion",
     "n8": "fritillary",
     "n9": "iris",
     "n10": "lily_valley ",
     "n11": "pansy",
     "n12": "snowdrop",
     "n13": "sunflower",
     "n14": "tigerlily",
     "n15": "tulip",
     "n16": "windflower"
}


def draw_test(name, pred, im):
    flower = "Predicted: "+ flower_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, flower, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
#     print(str(path_class))
    #print("Class - " + flower_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
#     print(file_path+"/"+image_name)
    return (file_path+"/"+image_name,   cv2.imread(file_path+"/"+image_name) )

for i in range(0,17):
    imagename, input_im = getRandomImage("./17_flowers/validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (img_rows, img_cols), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,img_rows,img_cols,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
#     print(str(res))
    
    # Show image with predicted class
    draw_test(imagename, res, input_original) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()


[9]
[2]
[9]
[7]
[11]
[13]
[2]
[13]
[9]
[16]
[13]
[7]
[5]
[0]
[9]
[9]
[16]
